In [1]:
import pandas as pd
import numpy as np
import re
import os
from Bio import Align
from Bio.Align import substitution_matrices
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from aaindex import aaindex1
import MDAnalysis as mda
from MDAnalysis.lib.distances import distance_array
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.autograd import Variable
from torch.autograd import grad as torch_grad
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

In [2]:
def parse_fasta_files(directory):
    data = []
    
    for filename in os.listdir(directory):
        if filename.endswith('.txt') and not filename.endswith('_neg.txt'):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r') as file:
                sequences = []
                current_sequence = ""
                for line in file:
                    line = line.strip()
                    if line.startswith("#"):
                        continue  # Ignore lines that start with #
                    if line.startswith(">"):
                        if current_sequence:
                            sequences.append(current_sequence)
                            current_sequence = ""
                    else:
                        current_sequence = line
                if current_sequence:
                    sequences.append(current_sequence)

                if sequences:
                    primary_sequence = sequences[0]
                    cross_reactive_sequences = sequences[1:]

                    # Add positive examples
                    for seq in cross_reactive_sequences:
                        data.append((filepath, primary_sequence, seq, 1))

                    # Add negative examples (assuming they are stored in a similar fashion)
                    non_cross_reactive_filename = filename.replace('.txt', '_neg.txt')
                    non_cross_reactive_filepath = os.path.join(directory, non_cross_reactive_filename)
                    if os.path.exists(non_cross_reactive_filepath):
                        with open(non_cross_reactive_filepath, 'r') as neg_file:
                            neg_sequences = []
                            current_sequence = ""
                            for line in neg_file:
                                line = line.strip()
                                if line.startswith("#"):
                                    continue  # Ignore lines that start with #
                                if line.startswith(">"):
                                    if current_sequence:
                                        neg_sequences.append(current_sequence)
                                        current_sequence = ""
                                else:
                                    current_sequence = line
                            if current_sequence:
                                neg_sequences.append(current_sequence)

                            for seq in neg_sequences:
                                data.append((non_cross_reactive_filepath, primary_sequence, seq, 0))
    
    return data

def read_and_combine_excels(directory):
    # List to hold dataframes
    df_list = []
    
    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.xls') or filename.endswith('.xlsx'):
            filepath = os.path.join(directory, filename)
            # Read the Excel file
            df = pd.read_excel(filepath, header=1)
            # Append the dataframe to the list
            df_list.append(df)
    
    # Combine all dataframes into one
    combined_df = pd.concat(df_list, ignore_index=True)
    
    return combined_df

def find_best_variant(sequence, df):
    variants = [
        sequence[:9],  # First 9 letters
        sequence[1:]   # Last 9 letters
    ]
    filtered_df = df[(df['Peptide'].isin(variants)) & (~df['Peptide'].str.contains('X'))]
    if filtered_df.empty:
        return None
    best_variant = filtered_df.loc[filtered_df['BA_Rank'].idxmin(), 'Peptide']
    return best_variant

def align_sequences(seq1, seq2):
    aligner = Align.PairwiseAligner()
    aligner.substitution_matrix = substitution_matrices.load("BLOSUM50")
    aligner.open_gap_score = -0.5
    aligner.extend_gap_score = -0.1
    alignments = aligner.align(seq1, seq2)
    best_alignment = alignments[0]
    
    # Extracting aligned sequences using indices
    aligned_seq1 = best_alignment.target
    aligned_seq2 = best_alignment.query
    start = best_alignment.aligned[0][0][0]  # Start of alignment in seq1
    
    return aligned_seq1, aligned_seq2, start

def get_best_subsequence(seq, length):
    best_subseq = None
    best_score = float('inf')
    for i in range(len(seq) - length + 1):
        subseq = seq[i:i + length]
        if subseq.find('X') == -1:  # Exclude sequences containing 'X'
            filtered_df = df[df['Peptide'] == subseq]
            if not filtered_df.empty:
                score = filtered_df.iloc[0]['BA_Rank']
                if score < best_score:
                    best_score = score
                    best_subseq = subseq
    return best_subseq

def process_epitopes(epitope_list, df):
    processed_list = []
    
    for filename, first_seq, second_seq, label in epitope_list:
        # Case 1: Both first and second sequences are 10 letters
        if len(first_seq) == 10 and len(second_seq) == 10:
            best_first_variant = find_best_variant(first_seq, df)
            if best_first_variant:
                pos_trim = first_seq.index(best_first_variant)
                best_second_variant = second_seq[pos_trim:pos_trim + 9]
                processed_list.append((filename, best_first_variant, best_second_variant, label))
        
        # Case 2: First sequence is 9 letters, second sequence is 10 letters
        elif len(first_seq) == 9 and len(second_seq) == 10:
            aligned_seq1, aligned_seq2, start = align_sequences(first_seq, second_seq)
            best_second_variant = aligned_seq2[start:start + 9]
            if not best_second_variant:
                best_second_variant = second_seq[:9]  # Default to first 9 letters if no alignment
            processed_list.append((filename, first_seq, best_second_variant, label))
        
        # Case 3: First sequence is 10 letters, second sequence is 9 letters
        elif len(first_seq) == 10 and len(second_seq) == 9:
            aligned_seq1, aligned_seq2, start = align_sequences(first_seq, second_seq)
            best_first_variant = aligned_seq1[start:start + 9]
            if not best_first_variant:
                best_first_variant = find_best_variant(first_seq, df)
            if best_first_variant:
                processed_list.append((filename, best_first_variant, second_seq, label))
        
        # Case 4: Both first and second sequences are 9 letters
        elif len(first_seq) == 9 and len(second_seq) == 9:
            processed_list.append((filename, first_seq, second_seq, label))
    
    return processed_list


# Load BLOSUM50 matrix
blosum50 = substitution_matrices.load("BLOSUM50")

hydrophobicity_values = aaindex1['KYTJ820101']

# Function to compute BLOSUM50 score for each position between two sequences
def compute_blosum50_scores(seq1, seq2):
    scores = []
    for aa1, aa2 in zip(seq1, seq2):
        score = blosum50.get((aa1, aa2), 0)  # Get the score for the pair (aa1, aa2)
        scores.append(score)
    return scores

# Function to calculate the alignment score
def calculate_blosum50_score_whole(seq1, seq2):
    aligner = Align.PairwiseAligner()
    aligner.substitution_matrix = blosum50
    alignment = aligner.align(seq1, seq2)
    # Return the score of the first alignment
    return alignment.score

# Function to filter sequences to standard amino acids only
def filter_standard_amino_acids(seq):
    standard_amino_acids = set('ACDEFGHIKLMNPQRSTVWY')
    return all(aa in standard_amino_acids for aa in seq)

# Function to compute hydrophobicity index using AAindex values
def compute_hydrophobicity(seq):
    return sum(hydrophobicity_values['values'][aa] for aa in seq) / len(seq)

# Function to compute isoelectric point
def compute_isoelectric_point(seq):
    analysis = ProteinAnalysis(seq)
    return analysis.isoelectric_point()

# Corrected add_scores_to_df function
def add_scores_to_df(df):
    blosum_scores = []
    hydrophobicity_diffs = []
    isoelectric_point_diff = []
    hydrophobicity_per_aa = []
    
    filtered_df = df[df.apply(lambda row: filter_standard_amino_acids(row['First Sequence']) and filter_standard_amino_acids(row['Second Sequence']), axis=1)].copy()
    
    for _, row in filtered_df.iterrows():
        seq1, seq2 = row['First Sequence'], row['Second Sequence']
        
        # Compute BLOSUM50 scores
        blosum_score = compute_blosum50_scores(seq1, seq2)
        blosum_scores.append(blosum_score)
        
        # Compute hydrophobicity index
        hydrophobicity_seq1 = compute_hydrophobicity(seq1)
        hydrophobicity_seq2 = compute_hydrophobicity(seq2)
        hydrophobicity_diff = hydrophobicity_seq1 - hydrophobicity_seq2
        
        # Compute hydrophobicity per amino acid differences
        hydrophobicity_per_aa_diff = [hydrophobicity_values['values'][aa1] - hydrophobicity_values['values'][aa2] for aa1, aa2 in zip(seq1, seq2)]
        
        hydrophobicity_diffs.append(hydrophobicity_diff)
        hydrophobicity_per_aa.append(hydrophobicity_per_aa_diff)
        
        # Compute isoelectric point difference
        isoelectric_point_diff.append(compute_isoelectric_point(seq1) - compute_isoelectric_point(seq2))
    
    # Convert list of scores to a DataFrame
    blosum_scores_df = pd.DataFrame(blosum_scores, columns=[f'BLOSUM50_Score_{i+1}' for i in range(len(blosum_scores[0]))])
    hydrophobicity_per_aa_df = pd.DataFrame(hydrophobicity_per_aa, columns=[f'Hydrophobicity_Diff_AA_{i+1}' for i in range(len(hydrophobicity_per_aa[0]))])
    
    filtered_df['Hydrophobicity Diff'] = hydrophobicity_diffs
    filtered_df['Isoelectric Point Diff'] = isoelectric_point_diff
    
    # Concatenate with the filtered DataFrame
    filtered_df = pd.concat([filtered_df, blosum_scores_df, hydrophobicity_per_aa_df], axis=1)

    return filtered_df


# Function to handle duplicate pairs
def process_group(group):
    # If all labels are the same, keep one instance
    if group['Label'].nunique() == 1:
        return group.iloc[0]
    # If labels are different, keep one instance with label 1
    else:
        return group[group['Label'] == 1].iloc[0]
    
def align_sequences_score(seq1, seq2, matrix_name="BLOSUM50"):
    # Load the BLOSUM62 matrix
    matrix = substitution_matrices.load(matrix_name)
    
    # Create an aligner object
    aligner = Align.PairwiseAligner()
    aligner.substitution_matrix = matrix
    
    # Perform the alignment
    score = aligner.score(seq1, seq2)

    return score

# Function to save DataFrames
def save_dataframes(group_name, df_include, df_exclude):
    include_filename = f'{group_name}_include.csv'
    exclude_filename = f'{group_name}_exclude.csv'
    df_include.to_csv(include_filename, index=False)
    df_exclude.to_csv(exclude_filename, index=False)
    print(f'{include_filename} and {exclude_filename} saved.')

def get_best_pdb_file(case_dir):
    # Initialize variables to track the best pdb
    min_value = float('inf')
    best_pdb = None
    
    # Construct the path to the molpdf_DOPE.tsv file
    tsv_path = os.path.join(case_dir, 'molpdf_DOPE.tsv')
    
    # Read the tsv file
    try:
        with open(tsv_path, 'r') as file:
            for line in file:
                parts = line.strip().split('\t')
                if len(parts) < 3:
                    continue
                score = float(parts[2])  # Convert the third column to float
                if score < min_value:
                    min_value = score
                    best_pdb = parts[0]  # Get the pdb filename
    except FileNotFoundError:
        print(f"File not found: {tsv_path}")
    
    return best_pdb

# Define a function to calculate distance differences
def calculate_distance_differences(row):
    first_peptide = row['First Sequence']
    second_peptide = row['Second Sequence']
    differences = {}
    
    # Ensure both peptides have distance data available
    if first_peptide in peptide_distances and second_peptide in peptide_distances:
        first_distances = peptide_distances[first_peptide]
        second_distances = peptide_distances[second_peptide]
        
        for key in first_distances:
            if key in second_distances:  # Ensure the same residue pair exists for both peptides
                # Calculate the difference in distances
                diff = first_distances[key] - second_distances[key]
                differences[key + ' Diff'] = diff
    
    return pd.Series(differences)

def parse_netmhcstabpan_output(file_path):
    hla_peptide_dict = {}
    current_hla = None
    
    try:
        with open(file_path, 'r') as file:
            for line in file:
                # Check for HLA line
                hla_match = re.search(r'HLA-(A|B)\*(\d+:\d+)', line)
                if hla_match:
                    hla_type = hla_match.group(1)
                    hla_number = hla_match.group(2).replace(':', '')
                    current_hla = f'HLA{hla_type}{hla_number}'
                
                # Check for peptide data line
                if line.startswith('    0  HLA-'):
                    parts = line.split()
                    if len(parts) >= 6:
                        peptide = parts[2]
                        rank_stab = parts[6]
                        
                        if current_hla:
                            key = f"{current_hla}, {peptide}"
                            hla_peptide_dict[key] = rank_stab

    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
    except IOError:
        print(f"Error: Unable to read file at {file_path}")
    
    return hla_peptide_dict

def add_stab_diff_column(df, netmhc_dict, column_name='Stability Diff'):
    def get_netmhc_diff(row):
        hla = row['HLA Type']
        seq1 = row['First Sequence']
        seq2 = row['Second Sequence']
        
        key1 = f"{hla}, {seq1}"
        key2 = f"{hla}, {seq2}"
        
        if key1 in netmhc_dict and key2 in netmhc_dict:
            return float(netmhc_dict[key1]) - float(netmhc_dict[key2])
        else:
            return None

    df[column_name] = df.apply(get_netmhc_diff, axis=1)
    return df

In [3]:
Ishizuka_HLAA2_pdf = pd.read_excel('CrossReactivityAssays/jimmunol0901607-sup-suppdata_removed.xlsx', header=16)

# Rename the columns
Ishizuka_HLAA2_pdf.rename(columns={
    'Se t': 'Set',
    'Pe ptide': 'Peptide',
    'Se que n ce': 'Sequence',
    'Organ ism': 'Organism',
    'Pro te in': 'Protein',
    'Unnamed: 5': 'Col5',
    'H LA A*0 2 0 1': 'HLA_A0201'
}, inplace=True)

In [4]:
# Drop unwanted columns
Ishizuka_HLAA2_pdf.drop(columns=['Set', 'Peptide', 'Col5', 'Organism', 'Protein'], inplace=True)

# Drop rows where 'Sequence' or 'HLA_A0201' is NaN
Ishizuka_HLAA2_pdf.dropna(subset=['Sequence', 'HLA_A0201'], inplace=True)

# Convert HLA_A0201 to numeric to handle any non-numeric issues
Ishizuka_HLAA2_pdf['HLA_A0201'] = pd.to_numeric(Ishizuka_HLAA2_pdf['HLA_A0201'], errors='coerce')

# Further drop rows where 'HLA_A0201' is NaN after conversion (if any)
Ishizuka_HLAA2_pdf.dropna(subset=['HLA_A0201'], inplace=True)

# Filter rows where 'HLA_A0201' is less than 500
Ishizuka_HLAA2_pdf = Ishizuka_HLAA2_pdf[Ishizuka_HLAA2_pdf['HLA_A0201'] >= 500]

Ishizuka_HLAA2_pdf = Ishizuka_HLAA2_pdf[Ishizuka_HLAA2_pdf['Sequence'].str.len().isin([9, 10])]


# Sequences to pair with
peptides = ['GILGFVFTL', 'SLFNTVATL', 'NLVPMVATV']

# Generate new pairs
new_pairs = []
for seq in Ishizuka_HLAA2_pdf['Sequence']:
    for peptide in peptides:
        new_pairs.append(['Ishizuka_HLAA0201', peptide, seq, 0])

In [5]:
fastas = parse_fasta_files('CrossReactivityAssays')
combined_netMHC = read_and_combine_excels('NetMHC')

In [7]:
# Convert each entry in new_pairs to a tuple and append to fastas
new_pairs_tuples = [tuple(pair) for pair in new_pairs]
fastas.extend(new_pairs_tuples)

In [8]:
# Process the list of epitopes
processed_epitopes = process_epitopes(fastas, combined_netMHC)
processed_epitopes_data = [(filename, first_seq, second_seq, label) for filename, first_seq, second_seq, label in processed_epitopes]
processed_epitopes_df = pd.DataFrame(processed_epitopes_data, columns=['Filename', 'First Sequence', 'Second Sequence', 'Label'])

# Filter rows where the sequence length is either 9 or 10
processed_epitopes_df = processed_epitopes_df[
    (processed_epitopes_df['First Sequence'].str.len().isin([9])) &
    (processed_epitopes_df['Second Sequence'].str.len().isin([9]))
]

In [9]:
processed_epitopes_df = add_scores_to_df(processed_epitopes_df)

# Extract the HLA type from the filename and create a new column 'HLA Type'
processed_epitopes_df['HLA Type'] = processed_epitopes_df['Filename'].str.extract(r'(HLA[A-Z0-9]+)')

# Drop the old 'Filename' column
#processed_epitopes_df = processed_epitopes_df.drop(columns=['Filename'])

processed_epitopes_df = processed_epitopes_df.dropna()

processed_epitopes_df = processed_epitopes_df[processed_epitopes_df['First Sequence'] != processed_epitopes_df['Second Sequence']]

In [10]:
# Create a new column combining 'First Sequence' and 'Second Sequence' in a tuple
processed_epitopes_df['Sequence_Pair'] = list(zip(processed_epitopes_df['First Sequence'], processed_epitopes_df['Second Sequence']))

# Find and print rows where 'Sequence_Pair' is duplicated
duplicate_pairs = processed_epitopes_df[processed_epitopes_df.duplicated('Sequence_Pair', keep=False)]

# Group by 'Sequence_Pair' and apply the function
processed_duplicates = duplicate_pairs.groupby('Sequence_Pair').apply(process_group).reset_index(drop=True)

# Combine processed duplicates with non-duplicates
non_duplicates = processed_epitopes_df.drop_duplicates('Sequence_Pair', keep=False)
final_df = pd.concat([non_duplicates, processed_duplicates], ignore_index=True)

# Drop the temporary 'Sequence_Pair' column if not needed
final_df = final_df.drop(columns=['Sequence_Pair'])

C:\Users\flo46\AppData\Local\Temp\ipykernel_11696\3010192585.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  processed_duplicates = duplicate_pairs.groupby('Sequence_Pair').apply(process_group).reset_index(drop=True)


In [11]:
# Create a dictionary to store NetMHCpan results for each HLA type
netmhcpan_results = {}

# Load the NetMHCpan results into the dictionary
for filename in os.listdir('NetMHCHLA'):
    if filename.endswith('.xlsx'):
        hla_type = filename.replace('.xlsx', '')
        filepath = os.path.join('NetMHCHLA', filename)
        df = pd.read_excel(filepath, header=1)
        netmhcpan_results[hla_type] = df.set_index('Peptide')

In [12]:
# Add new columns to the main DataFrame
final_df['First Sequence EL_Rank'] = 0.0
final_df['First Sequence BA_Rank'] = 0.0
final_df['Second Sequence EL_Rank'] = 0.0
final_df['Second Sequence BA_Rank'] = 0.0

In [13]:
# Iterate through the main DataFrame to look up ranks
for index, row in final_df.iterrows():
    hla_type = row['HLA Type']
    first_seq = row['First Sequence']
    second_seq = row['Second Sequence']

    # Look up EL_Rank and BA_Rank for the first sequence
    if first_seq in netmhcpan_results[hla_type].index:
        final_df.at[index, 'First Sequence EL_Rank'] = netmhcpan_results[hla_type].at[first_seq, 'EL_Rank']
        final_df.at[index, 'First Sequence BA_Rank'] = netmhcpan_results[hla_type].at[first_seq, 'BA_Rank']

    # Look up EL_Rank and BA_Rank for the second sequence
    if second_seq in netmhcpan_results[hla_type].index:
        final_df.at[index, 'Second Sequence EL_Rank'] = netmhcpan_results[hla_type].at[second_seq, 'EL_Rank']
        final_df.at[index, 'Second Sequence BA_Rank'] = netmhcpan_results[hla_type].at[second_seq, 'BA_Rank']

# Calculate the differences in EL_Rank and BA_Rank between first and second sequences
final_df['EL_Rank Diff'] = final_df['First Sequence EL_Rank'] - final_df['Second Sequence EL_Rank']
final_df['BA_Rank Diff'] = final_df['First Sequence BA_Rank'] - final_df['Second Sequence BA_Rank']


In [14]:
final_df['BLOSUM50_Alignment_Score'] = final_df.apply(lambda row: calculate_blosum50_score_whole(row['First Sequence'], row['Second Sequence']), axis=1)

In [68]:
def create_unique_sequence_hla_dataframe(df):
    # Extract and rename 'First Sequence' with 'HLA Type'
    first = df[['First Sequence', 'HLA Type']].rename(columns={'First Sequence': 'Peptide', 'HLA Type': 'Allele'})
    
    # Extract and rename 'Second Sequence' with 'HLA Type'
    second = df[['Second Sequence', 'HLA Type']].rename(columns={'Second Sequence': 'Peptide', 'HLA Type': 'Allele'})
    
    # Concatenate the two DataFrames vertically
    combined = pd.concat([first, second], axis=0)
    
    # Drop duplicates to get unique Peptide-Allele combinations
    unique_combinations = combined.drop_duplicates().reset_index(drop=True)
    
    # Add an 'ID' column starting from 1 up to the number of entries
    unique_combinations['ID'] = range(1, len(unique_combinations) + 1)
    
    # Rearrange columns to make 'ID' the first column
    cols = ['ID'] + [col for col in unique_combinations.columns if col != 'ID']
    unique_combinations = unique_combinations[cols]
    
    return unique_combinations

def convert_hla_syntax(hla):
    # Extract the letter and numbers
    match = re.match(r'HLA([AB])(\d{2})(\d{2})', hla)
    if match:
        letter, first_two, last_two = match.groups()
        return f"HLA-{letter}*{first_two}:{last_two}"
    else:
        return hla

In [61]:
pandora_df = create_unique_sequence_hla_dataframe(final_df)

In [69]:
pandora_df['Outdir'] = ['/mnt/c/Users/flo46/OneDrive/Dokumente/RP2_ML/PANDORA/Case' + str(i) for i in pandora_df['ID']]
pandora_df['Allele'] = pandora_df['Allele'].apply(convert_hla_syntax)

In [70]:
pandora_df

,ID,Peptide,Allele,Outdir
0,1,LAGIGILTV,HLA-A*02:01,/mnt/c/Users/flo46/OneDrive/Dokumente/RP2_ML/P...
1,2,YVLDHLIVV,HLA-A*02:01,/mnt/c/Users/flo46/OneDrive/Dokumente/RP2_ML/P...
2,3,AAGIGILTV,HLA-A*02:01,/mnt/c/Users/flo46/OneDrive/Dokumente/RP2_ML/P...
3,4,FMNKFIYEI,HLA-A*02:01,/mnt/c/Users/flo46/OneDrive/Dokumente/RP2_ML/P...
4,5,EVDPIGHLY,HLA-A*01:01,/mnt/c/Users/flo46/OneDrive/Dokumente/RP2_ML/P...
...,...,...,...,...
2734,2735,KTIAVSVYG,HLA-A*02:01,/mnt/c/Users/flo46/OneDrive/Dokumente/RP2_ML/P...
2735,2736,YCNYSKFWY,HLA-A*02:01,/mnt/c/Users/flo46/OneDrive/Dokumente/RP2_ML/P...
2736,2737,GLFNTVATL,HLA-A*02:01,/mnt/c/Users/flo46/OneDrive/Dokumente/RP2_ML/P...
2737,2738,SLFHTVATL,HLA-A*02:01,/mnt/c/Users/flo46/OneDrive/Dokumente/RP2_ML/P...


In [71]:
pandora_df.to_csv('pandora_input_long.csv', index=False)

### PYMOL ###

In [15]:
# Defining which MHC amino acids bind to which positions of the epitope
residue_interactions = {
    'A': {'residues': [5, 7, 59, 63, 66, 159, 163, 167, 171], 'binds_to': [1]},
    'B': {'residues': [7, 9, 24, 34, 45, 63, 66, 67, 70, 99], 'binds_to': [2]},
    'C': {'residues': [9, 70, 73, 74, 97], 'binds_to': [3, 5, 6]},
    'D': {'residues': [99, 114, 155, 156, 159, 160], 'binds_to': [3, 5, 6]},
    'E': {'residues': [97, 114, 147, 152, 156], 'binds_to': [5, 6]},
    'F': {'residues': [77, 80, 81, 84, 95, 123, 143, 146, 147], 'binds_to': [9]}
}

In [16]:
pandora_out_df = pd.read_csv('pandora_input_long.csv')
pandora_out_df.drop(pandora_out_df.columns[3], axis=1, inplace=True)

In [17]:
base_dir = 'PANDORA'

# Getting the pdb file with the best DOPE score in each PANDORA output

for case_id in range(1, 2740):  # Assuming case IDs go from 1 to 2725
    case_dir = os.path.join(base_dir, f'Case{case_id}', str(case_id))
    best_pdb = get_best_pdb_file(case_dir)
    
    # Update the DataFrame
    pandora_out_df.loc[pandora_out_df['ID'] == case_id, 'Best_PDB'] = best_pdb

In [18]:
# Initialize the overarching dictionary
pdb_distances = {}

# Calculate distances for each file using the residues defined earlier
for index, row in pandora_out_df.iterrows():
    case_id = row['ID']
    pdb_filename = row['Best_PDB']
    pdb_path = f"PANDORA/Case{case_id}/{case_id}/{pdb_filename}"

    # Initialize the dictionary for this PDB file
    if pdb_filename not in pdb_distances:
        pdb_distances[pdb_filename] = []

    # Load the PDB file
    u = mda.Universe(pdb_path)
    
    # Iterate through each interaction set in residue_interactions
    for key, interaction in residue_interactions.items():
        residues_M = interaction['residues']
        residues_P = interaction['binds_to']
        
        # Select residues in chain M and P
        atoms_M = u.select_atoms(f'segid M and resid {" ".join(map(str, residues_M))} and name CA')
        atoms_P = u.select_atoms(f'segid P and resid {" ".join(map(str, residues_P))} and name CA')
        
        # Calculate distances between each combination of residues
        for atom_M in atoms_M:
            for atom_P in atoms_P:
                distance = distance_array(atom_M.position, atom_P.position, box=u.dimensions)
                min_distance = distance.min()
                # Store the results in the dictionary
                pdb_distances[pdb_filename].append((f'Residue M {atom_M.resid}', f'Residue P {atom_P.resid}', min_distance))


In [19]:
# Initialize columns for each unique interaction
unique_interactions = set()
for interactions in pdb_distances.values():
    for interaction in interactions:
        unique_interactions.add(f'{interaction[0]} to {interaction[1]}')

# Add these as columns with default NaN values
for interaction in unique_interactions:
    pandora_out_df[interaction] = pd.NA

In [20]:
# Populate the DataFrame with distances
for pdb, interactions in pdb_distances.items():
    for interaction in interactions:
        interaction_label = f'{interaction[0]} to {interaction[1]}'
        # Find the row in the DataFrame where Best_PDB matches and update the distance
        pandora_out_df.loc[pandora_out_df['Best_PDB'] == pdb, interaction_label] = interaction[2]

In [21]:
peptide_distances = {}

for index, row in pandora_out_df.iterrows():
    peptide = row['Peptide']
    distances = {col: row[col] for col in pandora_out_df.columns if 'Residue' in col}
    peptide_distances[peptide] = distances

In [95]:
pandora_out_df.to_excel('pandora_out_long.xlsx')

In [22]:
# Apply the function to each row in final_df to calculate differences
distance_diffs = final_df.apply(calculate_distance_differences, axis=1)
final_df = pd.concat([final_df, distance_diffs], axis=1)

In [23]:
netMHCstab_dict = parse_netmhcstabpan_output('Immunogenicity/netMHCstabpan.txt')

# Initialize the dictionary to store the results
immune_dict = {}

current_allele = ""

# Open and read the file
with open('Immunogenicity/immunogenicity_out.txt', 'r') as file:
    for line_number, line in enumerate(file, 1):
        line = line.strip()
        if line.startswith("allele:"):
            # Extract the allele and format it
            current_allele = "HLA" + line.split()[1][3:].replace("-", "")
        elif line.startswith("masking") or line.startswith("masked variables"):
            # Skip these lines
            continue
        elif line.startswith("peptide,length,score"):
            # Skip the header line
            continue
        elif "," in line:
            # This is a data line
            parts = line.split(",")
            if len(parts) != 3:
                print(f"Warning: Unexpected format in line {line_number}: {line}")
                continue
            peptide, length, score = parts
            try:
                score = float(score)
                # Create the key as requested
                key = f"{current_allele}, {peptide}"
                # Store in the dictionary
                immune_dict[key] = score
            except ValueError:
                print(f"Warning: Invalid score in line {line_number}: {line}")

final_df = add_stab_diff_column(final_df, netMHCstab_dict, column_name='Stability Diff')
final_df = add_stab_diff_column(final_df, immune_dict, column_name='Immunogenicity Diff')

In [25]:
final_df.to_excel('features.xlsx')

In [24]:
final_df

,Filename,First Sequence,Second Sequence,Label,Hydrophobicity Diff,Isoelectric Point Diff,BLOSUM50_Score_1,BLOSUM50_Score_2,BLOSUM50_Score_3,BLOSUM50_Score_4,...,Residue M 63 to Residue P 2 Diff,Residue M 163 to Residue P 1 Diff,Residue M 74 to Residue P 5 Diff,Residue M 59 to Residue P 1 Diff,Residue M 114 to Residue P 5 Diff,Residue M 155 to Residue P 6 Diff,Residue M 152 to Residue P 5 Diff,Residue M 156 to Residue P 3 Diff,Stability Diff,Immunogenicity Diff
0,CrossReactivityAssays\Appay_HLAA0201.txt,LAGIGILTV,AAGIGILTV,1.0,0.222222,-0.045016,-2.0,5.0,8.0,5.0,...,0.007336,-0.055012,-1.166056,-0.051662,0.257933,1.258710,1.421522,0.460396,-1.0,0.00000
1,CrossReactivityAssays\Appay_HLAA0201.txt,LAGIGILTV,LAGIGTVPI,1.0,0.600000,0.000000,5.0,5.0,8.0,5.0,...,-0.000821,-0.025416,0.593583,-0.041323,-0.558375,0.762060,-0.167162,0.415955,-2.0,0.07370
2,CrossReactivityAssays\Appay_HLAA0201.txt,LAGIGILTV,VTGITIHFV,1.0,0.655556,-1.186116,1.0,0.0,8.0,5.0,...,-0.021605,-0.029176,-0.811462,-0.005329,0.927363,0.147223,3.319497,0.226172,4.5,-0.08718
3,CrossReactivityAssays\Appay_HLAA0201.txt,LAGIGILTV,VAGIGLLSV,1.0,0.044444,0.030011,1.0,5.0,8.0,5.0,...,0.008471,0.086436,-0.828199,-0.004476,0.479437,0.532198,1.466406,-0.049576,-3.0,0.25506
4,CrossReactivityAssays\Appay_HLAA0201.txt,LAGIGILTV,VAGIGILAI,1.0,-0.355556,0.030011,1.0,5.0,8.0,5.0,...,-0.010031,0.018301,-0.716859,-0.049965,-0.686382,1.089844,0.504220,0.438033,-10.0,-0.00018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5124,Ishizuka_HLAA0201,SLFNTVATL,YASIINRDI,0.0,1.000000,-0.595673,-4.0,-1.0,0.0,-4.0,...,0.076680,0.164161,-1.090366,-0.108695,1.470827,-0.056201,2.393111,0.286517,-24.0,-0.09652
5125,Ishizuka_HLAA0201,SLFNTVATL,YASLTTIGT,0.0,0.577778,-0.284309,0.0,-2.0,0.0,7.0,...,0.082487,0.229859,-2.438926,0.050313,0.297795,-2.276436,1.631423,0.107971,-23.0,0.02225
5126,Ishizuka_HLAA0201,SLFNTVATL,YAVLSEYET,0.0,1.333333,1.189981,-1.0,2.0,-4.0,-2.0,...,0.089126,0.310028,-2.751270,-0.404030,-0.360065,-1.807390,1.414340,-0.141591,-24.0,0.17308
5127,Ishizuka_HLAA0201,SLFNTVATL,YLSKEDRII,0.0,1.711111,-0.828997,1.0,1.0,-2.0,-4.0,...,0.069888,0.151065,-2.743226,-0.115433,-0.614903,-0.869662,1.265494,0.187427,-3.0,0.19473


### No Anchor Cross-Val Sets ###

In [36]:
final_df_noa = pd.read_excel('features_more.xlsx')
final_df_noa.drop('Unnamed: 0', axis=1, inplace=True)

In [37]:
columns_to_drop = ['BLOSUM50_Score_2', 'Hydrophobicity_Diff_AA_2', 'BLOSUM50_Score_9', 'Hydrophobicity_Diff_AA_9']

 # Drop specific columns
final_df_noa = final_df_noa.drop(columns=columns_to_drop, errors='ignore')
    
# Drop columns containing 'Residue P 2'
final_df_noa = final_df_noa.drop(columns=[col for col in final_df_noa.columns if 'Residue P 2' in col], errors='ignore')
final_df_noa = final_df_noa.drop(columns=[col for col in final_df_noa.columns if 'Residue P 9' in col], errors='ignore')

### Making the cross-validation sets ###

In [32]:
final_df = pd.read_excel('features_more.xlsx')
final_df.drop('Unnamed: 0', axis=1, inplace=True)
final_df_noa = pd.read_excel('features_no_anchor.xlsx')
final_df_noa.drop('Unnamed: 0', axis=1, inplace=True)

In [23]:
def group_sequences(sequences, threshold):
    aligner = Align.PairwiseAligner()
    groups = []
    
    for seq in sequences:
        found_group = False
        for group in groups:
            if any(aligner.align(seq, group_seq).score / len(seq) >= threshold for group_seq in group):
                group.append(seq)
                found_group = True
                break
        if not found_group:
            groups.append([seq])
    
    return groups

In [24]:
unique_sequences = final_df['First Sequence'].unique()

# 2. Create groups of sequences based on similarity
#similarity_threshold = 0.45  # This was for the short version
similarity_threshold = 0.8
sequence_groups = group_sequences(unique_sequences, similarity_threshold)

# 3. Print out the group sizes
for i, group in enumerate(sequence_groups, 1):
    group_size = final_df['First Sequence'].isin(group).sum()
    print(f"Group {i}: {group_size} sequences")
    #print("Example sequences:")
    #for seq in group[:3]:  # Print up to 3 example sequences per group
    #    print(f"  {seq}")
    print()

Group 1: 120 sequences

Group 2: 3 sequences

Group 3: 174 sequences

Group 4: 62 sequences

Group 5: 3 sequences

Group 6: 14 sequences

Group 7: 29 sequences

Group 8: 10 sequences

Group 9: 7 sequences

Group 10: 6 sequences

Group 11: 8 sequences

Group 12: 6 sequences

Group 13: 38 sequences

Group 14: 24 sequences

Group 15: 6 sequences

Group 16: 4 sequences

Group 17: 7 sequences

Group 18: 6 sequences

Group 19: 6 sequences

Group 20: 7 sequences

Group 21: 4 sequences

Group 22: 1 sequences

Group 23: 1 sequences

Group 24: 1 sequences

Group 25: 1 sequences

Group 26: 18 sequences

Group 27: 1 sequences

Group 28: 1 sequences

Group 29: 1 sequences

Group 30: 1595 sequences

Group 31: 10 sequences

Group 32: 1712 sequences

Group 33: 4 sequences

Group 34: 6 sequences

Group 35: 4 sequences

Group 36: 4 sequences

Group 37: 5 sequences

Group 38: 3 sequences

Group 39: 1 sequences

Group 40: 1 sequences

Group 41: 1 sequences

Group 42: 1 sequences

Group 43: 1 sequences

Gr

In [25]:
# 3. Define custom groupings
custom_groupings = {
    'Group_1_3': sequence_groups[0] + sequence_groups[2],
    'Group_30': sequence_groups[29],
    'Group_32': sequence_groups[31],
    'Group_47': sequence_groups[46],
    'Group_Rest': [seq for i, group in enumerate(sequence_groups) 
             if i not in [0, 2, 29, 31, 46] for seq in group]
}

for group_name, group_sequences in custom_groupings.items():
    # Create dataframe including rows with sequences from this group
    df_include = final_df[final_df['First Sequence'].isin(group_sequences)]
    
    # Create dataframe excluding rows with sequences from this group
    df_exclude = final_df[~final_df['First Sequence'].isin(group_sequences)]
    
    # Save dataframes as CSV files
    df_include.to_csv(f'{group_name}_include.csv', index=False)
    df_exclude.to_csv(f'{group_name}_exclude.csv', index=False)
    
    # Calculate the proportion of included to excluded
    included_count = len(df_include)
    excluded_count = len(df_exclude)
    proportion = included_count / excluded_count if excluded_count != 0 else float('inf')
    
    print(f"{group_name}:")
    print(f"  Included: {included_count}")
    print(f"  Excluded: {excluded_count}")
    print(f"  Proportion (Included/Excluded): {proportion:.4f}")
    print(f"  Saved {group_name}_include.csv and {group_name}_exclude.csv")
    print()

Group_1_3:
  Included: 294
  Excluded: 5164
  Proportion (Included/Excluded): 0.0569
  Saved Group_1_3_include.csv and Group_1_3_exclude.csv

Group_30:
  Included: 1595
  Excluded: 3863
  Proportion (Included/Excluded): 0.4129
  Saved Group_30_include.csv and Group_30_exclude.csv

Group_32:
  Included: 1712
  Excluded: 3746
  Proportion (Included/Excluded): 0.4570
  Saved Group_32_include.csv and Group_32_exclude.csv

Group_47:
  Included: 1528
  Excluded: 3930
  Proportion (Included/Excluded): 0.3888
  Saved Group_47_include.csv and Group_47_exclude.csv

Group_Rest:
  Included: 329
  Excluded: 5129
  Proportion (Included/Excluded): 0.0641
  Saved Group_Rest_include.csv and Group_Rest_exclude.csv



### Data Simulation ###

Targetting D loss of 0.5

In [3]:
final_df = pd.read_excel('features.xlsx')
final_df.drop('Unnamed: 0', axis=1, inplace=True)
final_df = final_df.drop(columns=['First Sequence', 'Second Sequence'])
final_df = pd.get_dummies(final_df, columns=['HLA Type'])

In [5]:
scaler = MinMaxScaler()

In [6]:
continuous_cols = [col for col in final_df.columns if re.search(r'(Diff|BA_rank|EL_rank)', col, re.I)]
final_df[continuous_cols] = scaler.fit_transform(final_df[continuous_cols])

In [8]:
# Identify discrete and continuous columns
discrete_cols = [col for col in final_df.columns if 'Label' in col or 'BLOSUM50' in col or 'HLA Type_' in col]
continuous_cols = [col for col in final_df.columns if col not in discrete_cols]
# Convert boolean columns to int
bool_columns = final_df.select_dtypes(include=['bool']).columns
final_df[bool_columns] = final_df[bool_columns].astype(int)

# Now convert to tensor
data_tensor = torch.FloatTensor(final_df.values)

In [12]:
class Generator(nn.Module):
    def __init__(self, latent_dim, output_dim, num_classes):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        self.output_dim = output_dim
        self.num_classes = num_classes

        self.label_emb = nn.Embedding(num_classes, num_classes)
        
        self.model = nn.Sequential(
            nn.Linear(latent_dim + num_classes, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, output_dim)
        )

    def forward(self, z, labels):
        c = self.label_emb(labels)
        x = torch.cat([z, c], 1)
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(Discriminator, self).__init__()
        self.label_emb = nn.Embedding(num_classes, num_classes)
        
        self.model = nn.Sequential(
            nn.Linear(input_dim + num_classes, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x, labels):
        c = self.label_emb(labels)
        x = torch.cat([x, c], 1)
        return self.model(x)

def train_cgan(data_tensor, labels_tensor, latent_dim, n_epochs, batch_size, lr):
    num_classes = len(torch.unique(labels_tensor))
    generator = Generator(latent_dim, data_tensor.shape[1], num_classes)
    discriminator = Discriminator(data_tensor.shape[1], num_classes)

    g_optimizer = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    d_optimizer = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

    criterion = nn.BCELoss()

    for epoch in range(n_epochs):
        for i in range(0, data_tensor.shape[0], batch_size):
            batch = data_tensor[i:i+batch_size]
            labels = labels_tensor[i:i+batch_size]

            # Train Discriminator
            d_optimizer.zero_grad()

            # Real data
            real_output = discriminator(batch, labels)
            d_real_loss = criterion(real_output, torch.ones_like(real_output))

            # Fake data
            z = torch.randn(batch.shape[0], latent_dim)
            fake_data = generator(z, labels)
            fake_output = discriminator(fake_data.detach(), labels)
            d_fake_loss = criterion(fake_output, torch.zeros_like(fake_output))

            d_loss = d_real_loss + d_fake_loss
            d_loss.backward()
            d_optimizer.step()

            # Train Generator
            g_optimizer.zero_grad()
            z = torch.randn(batch.shape[0], latent_dim)
            fake_data = generator(z, labels)
            fake_output = discriminator(fake_data, labels)
            g_loss = criterion(fake_output, torch.ones_like(fake_output))
            g_loss.backward()
            g_optimizer.step()

        if (epoch + 1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{n_epochs}], D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")

    return generator

# Prepare data
data_tensor = torch.FloatTensor(final_df.values)
labels_tensor = torch.LongTensor(final_df['Label'].values)

# Train the CGAN
latent_dim = 100
n_epochs = 1000
batch_size = 64
lr = 0.0002

trained_generator = train_cgan(data_tensor, labels_tensor, latent_dim, n_epochs, batch_size, lr)

Epoch [100/1000], D Loss: 1.3679, G Loss: 0.7205
Epoch [200/1000], D Loss: 1.3677, G Loss: 0.7005
Epoch [300/1000], D Loss: 1.2149, G Loss: 0.9868
Epoch [400/1000], D Loss: 1.1256, G Loss: 0.9457
Epoch [500/1000], D Loss: 1.0415, G Loss: 1.7125
Epoch [600/1000], D Loss: 0.9522, G Loss: 1.6887
Epoch [700/1000], D Loss: 1.0597, G Loss: 1.5025
Epoch [800/1000], D Loss: 0.8949, G Loss: 1.5981
Epoch [900/1000], D Loss: 0.8270, G Loss: 1.8327
Epoch [1000/1000], D Loss: 0.7771, G Loss: 1.5989


In [41]:
def generate_synthetic_data(generator, num_samples, latent_dim, num_classes, scaler, original_df):
    with torch.no_grad():
        z = torch.randn(num_samples, latent_dim)
        labels = torch.randint(0, num_classes, (num_samples,))
        synthetic_data = generator(z, labels)

    # Convert to DataFrame
    synthetic_df = pd.DataFrame(synthetic_data.numpy(), columns=original_df.columns)

    # Handle HLA Type_ columns
    hla_cols = [col for col in original_df.columns if 'HLA Type_' in col]
    hla_data = synthetic_df[hla_cols].values
    max_indices = np.argmax(hla_data, axis=1)
    hla_one_hot = np.zeros_like(hla_data)
    hla_one_hot[np.arange(len(hla_data)), max_indices] = 1
    synthetic_df[hla_cols] = hla_one_hot

    # Handle other discrete columns (including Label and BLOSUM50)
    other_discrete_cols = ['Label'] + [col for col in original_df.columns if 'BLOSUM50' in col]
    for col in other_discrete_cols:
        synthetic_df[col] = synthetic_df[col].round().clip(original_df[col].min(), original_df[col].max())

    # Handle continuous columns
    continuous_cols = [col for col in original_df.columns if col not in hla_cols and col not in other_discrete_cols]
    
    # Only inverse transform the columns that were actually scaled
    scaled_cols = [col for col in continuous_cols if col in scaler.feature_names_in_]
    synthetic_df[scaled_cols] = scaler.inverse_transform(synthetic_df[scaled_cols])
    
    # For other continuous columns, just clip to the original range
    for col in continuous_cols:
        if col not in scaled_cols:
            synthetic_df[col] = synthetic_df[col].clip(original_df[col].min(), original_df[col].max())

    return synthetic_df

# Generate synthetic data
num_samples = 5000
num_classes = 2  # Assuming binary classification
synthetic_df = generate_synthetic_data(trained_generator, num_samples, latent_dim, num_classes, scaler, final_df)

In [42]:
def validate_synthetic_data(original_df, synthetic_df):
    # Compare distributions
    for col in original_df.columns:
        print(f"\nColumn: {col}")
        print("Original distribution:")
        print(original_df[col].value_counts(normalize=True).head())
        print("Synthetic distribution:")
        print(synthetic_df[col].value_counts(normalize=True).head())
    
    # Check correlations
    original_corr = original_df[continuous_cols].corr()
    synthetic_corr = synthetic_df[continuous_cols].corr()
    print("\nCorrelation difference:")
    print((original_corr - synthetic_corr).abs().mean().mean())

validate_synthetic_data(final_df, synthetic_df)


Column: Label
Original distribution:
Label
0    0.944226
1    0.055774
Name: proportion, dtype: float64
Synthetic distribution:
Label
1.0    0.5088
0.0    0.4912
Name: proportion, dtype: float64

Column: Hydrophobicity Diff
Original distribution:
Hydrophobicity Diff
0.617960    0.005428
0.607306    0.005053
0.356164    0.004866
0.482496    0.004866
0.535769    0.004679
Name: proportion, dtype: float64
Synthetic distribution:
Hydrophobicity Diff
-0.980428    0.0004
 2.636924    0.0002
-0.341747    0.0002
 3.726015    0.0002
 0.238492    0.0002
Name: proportion, dtype: float64

Column: Isoelectric Point Diff
Original distribution:
Isoelectric Point Diff
0.706316    0.130264
0.579806    0.073929
0.681872    0.063448
0.303191    0.015160
0.579865    0.014786
Name: proportion, dtype: float64
Synthetic distribution:
Isoelectric Point Diff
 1.092692    0.0004
-9.201756    0.0002
-0.175183    0.0002
-4.780387    0.0002
-5.558524    0.0002
Name: proportion, dtype: float64

Column: BLOSUM50_Sco

In [43]:
original_corr = final_df[continuous_cols].corr()
synthetic_corr = synthetic_df[continuous_cols].corr()

correlation_diff = (original_corr - synthetic_corr).abs()

In [44]:
# Get the upper triangle of the difference matrix (excluding diagonal)
upper_tri = correlation_diff.where(np.triu(np.ones(correlation_diff.shape), k=1).astype(bool))

# Stack the matrix to get a series with multi-index
stacked_diff = upper_tri.stack()

# Sort the differences in descending order
sorted_diff = stacked_diff.sort_values(ascending=False)

# Display the top N differences
N = 10  # Adjust this to show more or fewer pairs
print("Top contributors to correlation discrepancy:")
print(sorted_diff.head(N))

Top contributors to correlation discrepancy:
Second Sequence EL_Rank            Stability Diff                       1.439312
BA_Rank Diff                       Stability Diff                       1.311160
Residue M 66 to Residue P 2 Diff   Residue M 66 to Residue P 1 Diff     1.277792
Residue M 152 to Residue P 6 Diff  Residue M 70 to Residue P 6 Diff     1.209226
Residue M 70 to Residue P 6 Diff   Residue M 147 to Residue P 6 Diff    1.204478
Residue M 97 to Residue P 5 Diff   Residue M 73 to Residue P 6 Diff     1.189478
Residue M 159 to Residue P 6 Diff  Residue M 155 to Residue P 6 Diff    1.173343
EL_Rank Diff                       Stability Diff                       1.165798
Residue M 70 to Residue P 5 Diff   Residue M 159 to Residue P 5 Diff    1.149741
Residue M 156 to Residue P 5 Diff  Residue M 70 to Residue P 5 Diff     1.104869
dtype: float64


In [45]:
hla_columns = [col for col in synthetic_df.columns if col.startswith('HLA Type_')]

In [46]:
def get_hla_type(row):
    for col in hla_columns:
        if row[col] == 1:
            return col.split('_', 1)[1]  # Splits on the first underscore and returns the second part
    return 'Unknown'  # In case no column is 1, which should not happen

# Apply this function to each row
synthetic_df['HLA Type'] = synthetic_df.apply(get_hla_type, axis=1)

C:\Users\flo46\AppData\Local\Temp\ipykernel_9320\682144586.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  synthetic_df['HLA Type'] = synthetic_df.apply(get_hla_type, axis=1)


In [47]:
synthetic_df = synthetic_df.drop(columns=hla_columns)

In [48]:
columns = list(synthetic_df.columns)
# Find the index of the reference column
index = columns.index('Hydrophobicity_Diff_AA_9')

# Remove 'HLA Type' from the current position and insert it after the index found
columns.remove('HLA Type')
columns.insert(index + 1, 'HLA Type')

# Reassign the columns to the DataFrame
synthetic_df = synthetic_df[columns]

In [49]:
# Generate the sequence numbers
num_rows = len(synthetic_df)
first_seq = ['First_Simulated_' + str(i) for i in range(1, num_rows + 1)]
second_seq = ['Second_Simulated_' + str(i) for i in range(1, num_rows + 1)]

# Create new columns
synthetic_df['First Sequence'] = first_seq
synthetic_df['Second Sequence'] = second_seq

# Move the new columns to the first and second position
col_order = ['First Sequence', 'Second Sequence'] + [col for col in synthetic_df.columns if col not in ['First Sequence', 'Second Sequence']]
synthetic_df = synthetic_df[col_order]

In [50]:
synthetic_df.to_csv('synthtic_features.csv')